In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

learning_rate = 1e-4
keep_prob_rate = 0.7 #
max_epoch = 2000
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d

    return

def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool

    return

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784])/255.
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])

#  卷积层 1
## conv1 layer ##

W_conv1 =                       # patch 7x7, in size 1, out size 32
b_conv1 =
h_conv1 =                       # 卷积  自己选择 选择激活函数
h_pool1 =                       # 池化

# 卷积层 2
W_conv2 =                        # patch 5x5, in size 32, out size 64
b_conv2 =
h_conv2 =                        # 卷积  自己选择 选择激活函数
h_pool2 =                        # 池化

#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# 交叉熵函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    for i in range(max_epoch):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
        if i % 100 == 0:
            print(compute_accuracy(
                mnist.test.images[:1000], mnist.test.labels[:1000]))


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.reshape(-1, 784).astype('float32') / 255.
x_test = x_test.reshape(-1, 784).astype('float32') / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 2000

def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 定义模型结构
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.W_conv1 = weight_variable([5, 5, 1, 32])
        self.b_conv1 = bias_variable([32])
        self.W_conv2 = weight_variable([5, 5, 32, 64])
        self.b_conv2 = bias_variable([64])
        self.W_fc1 = weight_variable([7*7*64, 1024])
        self.b_fc1 = bias_variable([1024])
        self.W_fc2 = weight_variable([1024, 10])
        self.b_fc2 = bias_variable([10])
        self.dropout = tf.keras.layers.Dropout(keep_prob_rate)

    @tf.function
    def call(self, x, training=None):
        x = tf.reshape(x, [-1, 28, 28, 1])
        h_conv1 = tf.nn.relu(conv2d(x, self.W_conv1) + self.b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)
        h_conv2 = tf.nn.relu(conv2d(h_pool1, self.W_conv2) + self.b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, self.W_fc1) + self.b_fc1)
        h_fc1_drop = self.dropout(h_fc1, training=training)
        return tf.nn.softmax(tf.matmul(h_fc1_drop, self.W_fc2) + self.b_fc2)

model = MNISTModel()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# 创建数据集对象
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(100)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(100)

for epoch in range(max_epoch):
    for batch_xs, batch_ys in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(batch_xs, training=True)
            batch_ys = tf.cast(batch_ys, tf.float32)
            loss = tf.reduce_mean(-tf.reduce_sum(batch_ys * tf.math.log(predictions), axis=1))

        # 计算梯度
        gradients = tape.gradient(loss, model.trainable_variables)

        # 检查并处理 None 梯度
        grads_and_vars = []
        for grad, var in zip(gradients, model.trainable_variables):
            if grad is not None:
                grads_and_vars.append((grad, var))
            else:
                print(f"警告: 变量 {var.name} 的梯度为 None")

        # 应用梯度
        if grads_and_vars:
            optimizer.apply_gradients(grads_and_vars)
        #else:
            #print("警告: 没有有效的梯度可应用")

    if epoch % 100 == 0:
        test_loss = 0
        test_accuracy = 0
        for test_xs, test_ys in test_dataset:
            test_predictions = model(test_xs, training=False)
            test_ys = tf.cast(test_ys, tf.float32)
            test_loss += tf.reduce_mean(-tf.reduce_sum(test_ys * tf.math.log(test_predictions), axis=1))
            test_accuracy += tf.reduce_mean(tf.cast(tf.equal(tf.argmax(test_predictions, axis=1), tf.argmax(test_ys, axis=1)), tf.float32))
        test_loss /= len(test_dataset)
        test_accuracy /= len(test_dataset)
        print(f'Epoch {epoch}: Loss = {test_loss:.4f}, Accuracy = {test_accuracy:.4f}')

Epoch 0: Loss = 8.0714, Accuracy = 0.0863


KeyboardInterrupt: 

In [ ]:
#由于tensorflow2.x版本删除了tensorflow.examples模块，因此对下方读取数据集的模块进行修改。
import tensorflow as tf
import numpy as np

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 数据预处理
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 700

# 定义计算准确率的函数
def compute_accuracy(model, x, y):
    y_pre = model.predict(x, verbose=0)
    correct_prediction = np.equal(np.argmax(y_pre, 1), np.argmax(y, 1))
    accuracy = np.mean(correct_prediction.astype('float32'))
    return accuracy

# 定义模型
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (7, 7), activation='relu', input_shape=(28, 28, 1), padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
        tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(1 - keep_prob_rate),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 创建并训练模型
model = create_model()

for i in range(max_epoch):
    # 随机选择100个样本进行训练
    indices = np.random.choice(len(x_train), 100, replace=False)
    batch_xs = x_train[indices]
    batch_ys = y_train[indices]

    model.train_on_batch(batch_xs, batch_ys)

    if i % 100 == 0:
        acc = compute_accuracy(model, x_test[:1000], y_test[:1000])
        print(f"Epoch: {i}, Accuracy: {acc:.4f}")

Epoch: 0, Accuracy: 0.1540
Epoch: 100, Accuracy: 0.8760
Epoch: 200, Accuracy: 0.9250
Epoch: 300, Accuracy: 0.9410
Epoch: 400, Accuracy: 0.9460
Epoch: 500, Accuracy: 0.9600
Epoch: 600, Accuracy: 0.9670
